In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def display_images(images, titles, cmap=None, figsize=(15, 5)):
    
    if cmap is None:
        cmap = [None] * len(images)
    
    if cmap == 'viridis':
        cmap = ['viridis'] * len(images)
            
    if cmap == 'gray':
        cmap = ['gray'] * len(images)
        
    fig, ax = plt.subplots(ncols=len(images), figsize=figsize)
    for i in range(len(images)):
        ax[i].imshow(images[i], cmap=cmap[i])
        ax[i].set_title(f'{titles[i]}')
        ax[i].axis('off')
    plt.show()

def colorSpace(image, name):
    colorSpaceImage = cv2.cvtColor(image, name)
    c1, c2, c3 = cv2.split(colorSpaceImage)
    return c1, c2, c3

def display_histogram(channels, titles, colors, figsize=(15, 5)):
    fig, ax = plt.subplots(ncols=len(channels), figsize=figsize)
    for i in range(len(channels)):
        ax[i].hist(channels[i][channels[i]>0].flatten(), bins=256, color= colors[i])
        ax[i].set_title(f'histogram of {titles[i]}')
        # Remove ticks and labels
        ax[i].tick_params(axis="both", which="both", bottom=False, top=False, left=False, right=False, labelbottom=True, labelleft=False)
plt.tight_layout()  # Adjust layout to prevent overlapping
plt.show()

def apply_Otsu_threshold(image, channel):
    thresholdValue, threshold = cv2.threshold(channel, 0, 255, cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)
    masked_image = cv2.bitwise_and(image, image, mask=threshold.astype(np.uint8)*255)
    return thresholdValue, threshold, masked_image


<Figure size 640x480 with 0 Axes>

In [ ]:
#Post-processing the Masks
input_path = '/mnt/g/Data/BF_981'
mask_path = '/mnt/c/Users/praba/sciebo/Thesis/Results/Results/SW/Human'
output_path = '/mnt/c/Users/praba/sciebo/Thesis/Results/Results/SW/Human'
os.makedirs(output_path, exist_ok=True)
# seg_path = '/mnt/g/Result/predictions_100K_Random/newImages'
image_files, mask_files = sorted(os.listdir(input_path)), sorted(os.listdir(mask_path))
assert len(image_files) == len(mask_files)
data = pd.DataFrame({'image': image_files, 
                     'mask': mask_files})

for image_name, mask_name in zip(data['image'], data['mask']):
    # Load image and mask using OpenCV
    image = cv2.imread(os.path.join(input_path, image_name))
    mask = cv2.imread(os.path.join(mask_path, mask_name), 0)    
    segmented_image = cv2.bitwise_and(image, image, mask=mask.astype(np.uint8)*255)
    L, A, B = colorSpace(segmented_image, cv2.COLOR_RGB2LAB)
    value, threshold, masked_image = apply_Otsu_threshold(segmented_image, B)
    # cv2.imwrite(os.path.join(seg_path, image_name), masked_image)
    black_mask = np.all(masked_image == 0, axis=2)
    refined_mask = mask.copy()
    refined_mask[black_mask] = 0  
    cv2.imwrite(os.path.join(output_path, mask_name), refined_mask)

print(f'processed all images')


processed all images


In [ ]:
# #Segmentation of Block face images
# input_path = '/mnt/g/Data/BF_981'
# mask_path = '/mnt/d/Results/wd1e_6/newrandom/256/50K/pred'
# output_path = '/mnt/d/Results/wd1e_6/newrandom/256/50K/seg'

# os.makedirs(output_path, exist_ok=True)
# image_files, mask_files = sorted(os.listdir(input_path)), sorted(os.listdir(mask_path))
# assert len(image_files) == len(mask_files)
# data = pd.DataFrame({'image': image_files,
#                      'mask': mask_files})

# for image_name, mask_name in zip(data['image'], data['mask']):
#     # Load image and mask using OpenCV
#     image = cv2.imread(os.path.join(input_path, image_name))
#     mask = cv2.imread(os.path.join(mask_path, mask_name), 0)    
#     segmented_image = cv2.bitwise_and(image, image, mask=mask.astype(np.uint8)*255)
#     cv2.imwrite(os.path.join(output_path, mask_name), segmented_image)

# print(f'processed all images')
